<a href="https://colab.research.google.com/github/YunusKaratepe/duygu-tanima/blob/main/BP_Spectrogram_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spectrogram Extracting**

# Importing Libraries

In [235]:
!pip3 install SpecAugment
#!pip install tensorflow==1.15

import tensorflow
print(tensorflow.__version__)

import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import matplotlib
matplotlib.use('TKAgg')
import matplotlib.pyplot as plt

import os
from specAugment import spec_augment_tensorflow
from google.colab import drive
drive.mount("/content/drive")

1.15.0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Get Audio Dataset

In [236]:
def getAudios():
  audioArr = []

  directory = '/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2'

  i=0
  limit = 10

  for filename in os.listdir(directory):
    audioArr.append(directory + '/' + filename)
    #i += 1
    #if i > limit:
    #  break

  audioArr.sort() #Dosyalari isimlerine gore siralamak icin. Aslinda gerek yok, denemeler icin boyle.


  for filename in audioArr:
    print(filename)

  print()
  print()

  #audioArr.append(directory + '/03-01-01-01-01-01-02.wav')
  #audioArr.append(directory + '/03-01-01-01-01-02-02.wav')
  #audioArr.append(directory + '/03-01-01-01-02-01-02.wav')
  #audioArr.append(directory + '/03-01-01-01-02-02-02.wav')


  return audioArr

# Name Parser

In [237]:
def nameParser(name): #Spectrogramlarin kaydedileceği ismi düzenlemek için fonksiyon.
  directory = '/content/drive/My Drive/BP_DATASETS/MEL_SPECTROGRAMS/MEL_SPECTROGRAM_128'
  corePath = os.path.splitext(name)[0]
  splitArr = corePath.split('/')
  newName = directory + '/' + splitArr[len(splitArr) - 1] + '.jpeg'

  return newName

def nameParser2(name, directory): #Spectrogramlarin kaydedileceği ismi düzenlemek için fonksiyon.
  corePath = os.path.splitext(name)[0]
  splitArr = corePath.split('/')
  newName = directory + '/' + splitArr[len(splitArr) - 1] + '.jpeg'

  return newName


# Get Spectrogram


In [238]:
def getSpectrogram(audioArr, FRAME_SIZE = 2048, HOP_SIZE = 512):
  audios_librosa = []

  _, sr = librosa.load(audioArr[0])
  for audio in audioArr:
    print(audio)
    loadObj, _ = librosa.load(audio)  #Ses dosyasi librosa'ya yuklenir.
    stftObj = librosa.stft(loadObj, n_fft=FRAME_SIZE, hop_length=HOP_SIZE) #Yuklenen dosyaya st fourier transform uygulanir.
    dbObj = librosa.power_to_db(np.abs(stftObj) ** 2) #Power to DB araştırılacak. Neden yaptigimizi hatiramiyorum.
    audios_librosa.append(dbObj) #Ses datasının son hali bir listeye kaydedilir.

  return audios_librosa, sr    

# Plot Spectrogram

In [239]:
def plotSpectrogram(specList, nameArr, sr, hop_length = 512, y_axis="linear"):
  
  for specObj, name in zip(specList, nameArr):
    print(name)
    fileName = nameParser(name)
    print(fileName)

    plt.figure(figsize=(25, 10))
    librosa.display.specshow(specObj,                            
                              hop_length=hop_length, 
                              #x_axis="time", 
                              #y_axis=y_axis
                              sr=sr)
    plt.savefig(fileName) #Mel spectrogramı dosya sistemine kaydeder.
    plt.close() #Figurugu gostermesini engellemek icin.

    #plt.colorbar(format="%+2.f")
    #plt.show()

# Get Mel Spectrogram

In [240]:
def getMelSpectrogram(audioArr, FRAME_SIZE = 2048, HOP_SIZE = 512, N_MELS = 90):
  melSpectrogramList = []

  _, sr = librosa.load(audioArr[0])
  for audio in audioArr:
    print(audio)
    loadObj, _ =  librosa.load(audio)
    mel_spectrogram = librosa.feature.melspectrogram(loadObj, sr=sr, n_fft= FRAME_SIZE, hop_length= HOP_SIZE, n_mels= N_MELS)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    melSpectrogramList.append(log_mel_spectrogram)

  return melSpectrogramList, sr 

# Plot Mel Spectrogram

In [241]:
def plotMelSpectrogram(melSpecList, nameArr, sr, hop_length = 512):
  augmentationPath = '/content/drive/My Drive/BP_DATASETS/Augmented_Mel_Spectrograms/Normal_Augmentation_128'
  #path2 = '/content/drive/My Drive/BP_DATASETS/MEL_SPECTROGRAMS/MEL_SPECTROGRAM_128'
  
  for melObj, name in zip(melSpecList, nameArr):
    print(name)
    fileName = nameParser2(name, augmentationPath)
    print(fileName)
    plt.figure(figsize=(25, 10))
    librosa.display.specshow(melObj, 
                            x_axis="time",
                            y_axis="mel", 
                            sr=sr,
                            )
    plt.axis('off')
    plt.savefig(fileName, bbox_inches='tight', pad_inches = 0) #Mel spectrogramı dosya sistemine kaydeder.
    plt.close() #Figurugu gostermesini engellemek icin.
    #plt.colorbar(format="%+2.f")
    #plt.show()

# Derivative Augmentation

In [242]:
def derivativeAugmentation(melSpecList, order = 1):
  augmentedList = []
  for spec in melSpecList:
    derivatedSpectrogram = librosa.feature.delta(spec, order=order)
    augmentedList.append(derivatedSpectrogram)

  return augmentedList    

# Normal Augmentation 

In [243]:
def normalAugmentation(melSpecList):
  augmentedList = []
  i = 0
  for spec in melSpecList:
    newSpectrogram = spec_augment_tensorflow.spec_augment(mel_spectrogram=spec, time_warping_para=8, frequency_masking_para=27,
                                                                     time_masking_para=20, frequency_mask_num=1, time_mask_num=1)
    augmentedList.append(newSpectrogram)
    print(i)
    i+= 1

  return augmentedList    

# Verify Augmentation


In [244]:
def verifyAugmentation(audios):
  print("Verify Function")
  nameArr = []
  savedArr = []
  augmentationPath = '/content/drive/My Drive/BP_DATASETS/Augmented_Mel_Spectrograms/Normal_Augmentation_128'
  for audio in audios:
    corePath = os.path.splitext(str(audio))[0]
    splitArr = corePath.split('/')
    nameArr.append(splitArr[len(splitArr) - 1])

  for filename in os.listdir(augmentationPath):
    corePath = os.path.splitext(filename)[0]
    savedArr.append(corePath)

  for name in nameArr:
    if name not in savedArr:
      print(name)

# Main

In [245]:
FRAME_SIZE = 2048
HOP_SIZE = 512
N_MELS = 128 #Mel sayisi. Mel spectrogram icin lazim.

audios = getAudios() #Ses dosyalarinin adlari drive'dan alinip bu listeye kaydedildi.
print('Sesler okundu.')

#specList, sr = getSpectrogram(audios, HOP_SIZE = HOP_SIZE, FRAME_SIZE = FRAME_SIZE)
#plotSpectrogram(specList, audios, sr, hop_length = HOP_SIZE)

melSpecList, sr2 = getMelSpectrogram(audioArr = audios, FRAME_SIZE = FRAME_SIZE, HOP_SIZE = HOP_SIZE, N_MELS = N_MELS)
#plotMelSpectrogram(melSpecList, audios, sr2, HOP_SIZE)

#derivativeAugmentedMelList = derivativeAugmentation(melSpecList, order = 1)
#plotMelSpectrogram(derivativeAugmentedMelList, audios, sr2, HOP_SIZE)

#normalAugmentedList = normalAugmentation(melSpecList)
#plotMelSpectrogram(normalAugmentedList, audios, sr2, HOP_SIZE)

/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_01_00.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_01_01.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_01_02.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_01_03.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_02_04.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_02_05.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_02_06.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_02_07.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_02_08.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_02_09.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_02_10.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_02_11.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_03_12.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_03_13.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_03_14.wav
/content/drive/My Drive/BP_DATASETS/BP_Dataset_V2/01_03

# BOŞ

In [247]:
verifyAugmentation(audios)
#plotMelSpectrogram(normalAugmentedList[1036:], audios, sr2, HOP_SIZE)

Verify Function
